In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.feature_extraction.text import TfidfVectorizer

# Đọc dữ liệu
df = pd.read_csv('Data/trending_videos_unique.csv')

# Chuyển đổi 'trending_time' từ dạng chuỗi thành số ngày
df['trending_time_days'] = df['trending_time'].str.replace(' days', '').astype(int)

# Xử lý 'video_tags': Kết hợp các tag thành một chuỗi
df['video_tags_clean'] = df['video_tags'].fillna('').apply(lambda x: ' '.join(x.split(',')))

# Chuyển đổi 'duration' thành giây
df['duration_seconds'] = pd.to_timedelta(df['duration']).dt.total_seconds()

# Mã hóa 'category' bằng One-Hot Encoding
df = pd.get_dummies(df, columns=['category'], prefix='category')

# Mã hóa 'language' bằng Label Encoding
le = LabelEncoder()
df['language_encoded'] = le.fit_transform(df['langauge'].astype(str))

# Tạo các biến mới (Feature Engineering)
df['like_ratio'] = df['like_count'] / df['view_count']
df['comment_ratio'] = df['comment_count'] / df['view_count']

# Chọn các đặc tính và biến mục tiêu
feature_columns = [
    'view_count', 'like_count', 'comment_count', 'duration_seconds',
    'language_encoded', 'like_ratio', 'comment_ratio'
] + [col for col in df.columns if col.startswith('category_')]

X = df[feature_columns]
y = df['trending_time_days']

# Xử lý 'video_tags_clean' sử dụng TF-IDF
tag_vectorizer = TfidfVectorizer(max_features=100)
tag_tfidf = tag_vectorizer.fit_transform(df['video_tags_clean']).toarray()
tag_df = pd.DataFrame(tag_tfidf, columns=[f'tag_{i}' for i in range(tag_tfidf.shape[1])])

# Kết hợp các đặc tính
X = pd.concat([X.reset_index(drop=True), tag_df.reset_index(drop=True)], axis=1)

# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Khởi tạo mô hình Random Forest Regressor
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Huấn luyện mô hình
model.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = model.predict(X_test)

# Đánh giá mô hình
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error: {mae:.2f} days')
print(f'Mean Squared Error: {mse:.2f} days^2')
print(f'R^2 Score: {r2:.2f}')

# Ví dụ dự đoán thời gian trending cho một video mới
def predict_trending_time(view_count, like_count, comment_count, duration, language, category, tags):
    # Chuyển đổi duration thành giây
    duration_seconds = pd.to_timedelta(duration).total_seconds()
    
    # Mã hóa ngôn ngữ
    language_encoded = le.transform([language])[0]
    
    # Mã hóa category
    category_cols = [col for col in X.columns if col.startswith('category_')]
    category_encoded = [1 if col == f'category_{category}' else 0 for col in category_cols]
    
    # Xử lý tags
    tags_clean = ' '.join(tags.split(','))
    tags_tfidf = tag_vectorizer.transform([tags_clean]).toarray()[0]
    tags_encoded = tags_tfidf.tolist()
    
    # Tính tỷ lệ like và comment
    like_ratio = like_count / view_count
    comment_ratio = comment_count / view_count
    
    # Tạo DataFrame cho mẫu mới
    new_data = pd.DataFrame([[
        view_count, like_count, comment_count, duration_seconds,
        language_encoded, like_ratio, comment_ratio
    ] + category_encoded + tags_encoded], columns=X.columns)
    
    # Dự đoán
    predicted_trending_time = model.predict(new_data)[0]
    return predicted_trending_time

# Ví dụ sử dụng hàm dự đoán
predicted_days = predict_trending_time(
    view_count=10000000,
    like_count=100000,
    comment_count=100000,
    duration='0:05:00',
    language='vi',
    category='Music',
    tags='music,mtp'
)
print(f'Predicted Trending Time: {predicted_days:.2f} days')

Mean Absolute Error: 2.47 days
Mean Squared Error: 10.80 days^2
R^2 Score: 0.48
Predicted Trending Time: 17.23 days
